In [1]:
import pandas as pd
dados=pd.read_csv('C:/Users/Falcone/Documents/0_MachineLearning/2022.05.17_RandomForest/BRAZIL_CITIES.csv',sep=';')
#dados

In [2]:
dados.drop(['CITY', 'IDHM Ranking 2010', 'IDHM_Renda', 'IDHM_Longevidade', 'IDHM_Educacao', 'LONG', 'LAT', 'GVA_MAIN',
            'REGIAO_TUR', 'MUN_EXPENDIT', 'HOTELS', 'BEDS', 'Pr_Agencies', 'Pu_Agencies', 'Pr_Bank', 'Pu_Bank',
            'Pr_Assets', 'Pu_Assets','UBER','MAC','WAL-MART'], axis=1, inplace=True)

In [3]:
#dados.isnull().sum()/len(dados['STATE'])*100

In [4]:
#dados.dtypes

In [5]:
state_encode=pd.get_dummies(dados['STATE'])
categoria_encode=pd.get_dummies(dados['CATEGORIA_TUR'])
rural_encode=pd.get_dummies(dados['RURAL_URBAN'])
dados.drop('STATE', axis=1, inplace=True)
dados.drop('CATEGORIA_TUR', axis=1, inplace=True)
dados.drop('RURAL_URBAN', axis=1, inplace=True)
dados2=pd.concat([dados,state_encode,categoria_encode,rural_encode], axis=1)

In [6]:
#dados2

In [7]:
pd.set_option('display.max_rows',100)
dados2.isnull().sum()/len(dados2['CAPITAL'])*100
#A propria conversao de texto para numeros, eliminou os dados NaN que estavam na variavel CATEGORIA_TUR por exemplo

CAPITAL                    0.000000
IBGE_RES_POP               0.143549
IBGE_RES_POP_BRAS          0.143549
IBGE_RES_POP_ESTR          0.143549
IBGE_DU                    0.179437
IBGE_DU_URBAN              0.179437
IBGE_DU_RURAL              1.453436
IBGE_POP                   0.143549
IBGE_1                     0.143549
IBGE_1-4                   0.143549
IBGE_5-9                   0.143549
IBGE_10-14                 0.143549
IBGE_15-59                 0.143549
IBGE_60+                   0.143549
IBGE_PLANTED_AREA          0.053831
IBGE_CROP_PRODUCTION_$     0.053831
IDHM                       0.143549
ALT                        0.161493
PAY_TV                     0.053831
FIXED_PHONES               0.053831
AREA                       0.053831
ESTIMATED_POP              0.053831
GVA_AGROPEC                0.053831
GVA_INDUSTRY               0.053831
GVA_SERVICES               0.053831
GVA_PUBLIC                 0.053831
 GVA_TOTAL                 0.053831
TAXES                      0

In [8]:
dados2.dropna(axis=0, inplace=True)

In [9]:
pd.set_option('display.max_rows',100)
dados2.isnull().sum()/len(dados2['CAPITAL'])*100

CAPITAL                    0.0
IBGE_RES_POP               0.0
IBGE_RES_POP_BRAS          0.0
IBGE_RES_POP_ESTR          0.0
IBGE_DU                    0.0
IBGE_DU_URBAN              0.0
IBGE_DU_RURAL              0.0
IBGE_POP                   0.0
IBGE_1                     0.0
IBGE_1-4                   0.0
IBGE_5-9                   0.0
IBGE_10-14                 0.0
IBGE_15-59                 0.0
IBGE_60+                   0.0
IBGE_PLANTED_AREA          0.0
IBGE_CROP_PRODUCTION_$     0.0
IDHM                       0.0
ALT                        0.0
PAY_TV                     0.0
FIXED_PHONES               0.0
AREA                       0.0
ESTIMATED_POP              0.0
GVA_AGROPEC                0.0
GVA_INDUSTRY               0.0
GVA_SERVICES               0.0
GVA_PUBLIC                 0.0
 GVA_TOTAL                 0.0
TAXES                      0.0
GDP                        0.0
POP_GDP                    0.0
GDP_CAPITA                 0.0
COMP_TOT                   0.0
COMP_A  

In [10]:
dadosx=dados2.drop('IDHM', axis=1)
dadosy=dados2['IDHM']

In [11]:
#dadosx

In [12]:
#dadosy

In [13]:
from sklearn.preprocessing import MinMaxScaler
normaliza=MinMaxScaler(feature_range=(0, 1))
dadosxnorm=normaliza.fit_transform(dadosx)

In [14]:
#dadosxnorm

In [15]:
from sklearn.decomposition import PCA
pca=PCA(n_components=15)
dadosx_pca=pca.fit_transform(dadosxnorm)

In [16]:
#dadosx_pca

In [18]:
pca.explained_variance_ratio_

array([0.19583701, 0.12162792, 0.08095638, 0.07072222, 0.05173514,
       0.05057867, 0.04195469, 0.0390812 , 0.03509223, 0.03082312,
       0.0295308 , 0.02239032, 0.02061981, 0.01971799, 0.01785609])

In [19]:
sum(pca.explained_variance_ratio_)

0.8285235739213861

In [20]:
#rodar o modelo RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
modelo=RandomForestRegressor(n_estimators=100)
kfold=KFold(n_splits=3)
resultado=cross_val_score(modelo,dadosx_pca,dadosy,cv=kfold,n_jobs=-1)
resultado.mean()

0.8027482936840076

In [21]:
resultado=cross_val_score(modelo,dadosxnorm,dadosy,cv=kfold,n_jobs=-1)
resultado.mean()

0.8678323786491414

In [31]:
from sklearn.model_selection import GridSearchCV
param={'max_depth':[15,20,30,50],'min_samples_split':[2,3,4,5,7,9,11],'min_samples_leaf':[3,5,6,9,11]}
grid_randomforest=GridSearchCV(modelo,param,cv=3,n_jobs=-1)
grid_randomforest.fit(dadosx_pca,dadosy)
print(grid_randomforest.best_params_)
print(grid_randomforest.best_score_)

{'max_depth': 15, 'min_samples_leaf': 5, 'min_samples_split': 2}
0.808254856996164
